<a href="https://colab.research.google.com/github/mbc2009/Lmp_ML/blob/main/Model_trainning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Enviornment Initialization

In [1]:
%%capture
%%bash

# remove unnecessary
rm -rf *

# update pip
python -m pip install --upgrade pip

# install package
pip install opencv-python pillow
pip install segmentation_models_pytorch
pip install -q kaggle
pip install dropbox
pip install scikit-image

In [2]:
# basic import
import  os, sys, time, math, random, math, psutil, h5py, re
from    typing                  import  List, Tuple
from    dropbox                 import  Dropbox
from    tqdm                    import  tqdm
from    matplotlib              import  pyplot      as plt
import  numpy                                       as np
import  pandas                                      as pd
import  zipfile
import  warnings
import  shutil

from    skimage                 import  io
from    scipy                   import  interpolate
from    scipy.interpolate       import  RegularGridInterpolator
from    scipy.ndimage           import  generic_filter
from    PIL                     import  Image

import  torch
from    torch                   import  nn
from    torch.nn                import  functional  as F
import  torch.optim                                 as optim
import  torchvision.transforms.functional           as TF
from    torch.utils.data        import  Dataset, DataLoader, TensorDataset, random_split, Subset
from    torchvision             import  transforms, models
from    torchvision.transforms  import  *
import  kagglehub

In [3]:
# check hardware
print(f"CPU core #:\t{os.cpu_count()}")
print(f"CPU threads #:\t{psutil.cpu_count(logical=True)}")
print(f"Total memory:\t\t{psutil.virtual_memory().total / (1024**3):.2f} GB")
if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    print(f"available GPU #:\t{gpu_count}")
    for i in range(gpu_count):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i+1}:\t\t{gpu_name}")
else:
    print("No available GPU")

CPU core #:	2
CPU threads #:	2
Total memory:		12.67 GB
No available GPU


# 1. Configuration

In [14]:
class Confuration():
  # define paths to directory
  home_dir                = os.path.expanduser("~")
  working_dir             = os.path.join(os.getcwd(),'Lmp_ML')
  DataBase_dir            = os.path.join(working_dir,'DataBase')
  DB_3D_Grids_path        = os.path.join(DataBase_dir, '4', '3D_Grids.h5') # TODO: choose database version, here ver=4

conf = Confuration()

In [5]:
# import code from git hub
!git clone https://github.com/mbc2009/Lmp_ML

Cloning into 'Lmp_ML'...
remote: Enumerating objects: 511, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 511 (delta 21), reused 11 (delta 4), pack-reused 450 (from 3)
Receiving objects: 100% (511/511), 128.60 MiB | 9.28 MiB/s, done.
Resolving deltas: 100% (237/237), done.


In [6]:
# make directory
os.makedirs(conf.DataBase_dir, exist_ok=True)

In [7]:
# download
original_path = kagglehub.dataset_download("mbc2009/heat-and-mass-transfer",force_download=True)
shutil.move(original_path, conf.DataBase_dir)

100%|██████████| 773M/773M [00:19<00:00, 41.4MB/s]

Extracting files...


'/content/Lmp_ML/DataBase/4'

# 2.Functions and Classes

In [ ]:
def check_hdf5_content(file_path:str,PrintStrcut=False)->int:
  '''
  check the content (name, quantity) of hdf5 file
  input:
    file_path: the path of hdf5 file
    PrintStrcut: print the structure of hdf5 file
  return:
    the name and quantity of variables in hdf5 file
  '''
  # 初始化数据集计数器
  dataset_count = 0

  # 定义一个内部函数用于遍历 HDF5 文件内部
  def count_datasets(name, obj):
        nonlocal dataset_count
        if isinstance(obj, h5py.Dataset):  # 判断是否为数据集
            dataset_count += 1
        elif isinstance(obj, h5py.Group):  # 判断是否为组
            pass  # 如果是组，不计数

  with h5py.File(file_path, "r") as h5f:

        # 遍历文件内容以计数
        h5f.visititems(count_datasets)

        # 打印所有内变量名字
        if PrintStrcut:
          print(f"文件结构:")
          h5f.visit(print)

  print(f'文件数: {dataset_count}')

  return dataset_count # 文件数


# 3.Prepare Data Base

## 3.1.Check grid data base

In [29]:
# 查看hdf5文件内容
num_grids = check_hdf5_content(conf.DB_3D_Grids_path,PrintStrcut=False)

文件数: 836


In [45]:
# 查看单个数据点
DB_3D_Grids = h5py.File(conf.DB_3D_Grids_path, "r")
GA          = DB_3D_Grids[f"len_{2}_sigma_{8}_{343}"][:]
print(f'矩阵形状: {GA.shape}')
print(f'矩阵最大值: {np.max(GA)}')
print(f'矩阵最小值: {np.min(GA)}')
print(f"矩阵 GA 中 {np.max(GA)} 的数量：{np.count_nonzero(GA == np.max(GA)) }")

矩阵形状: (400, 400, 400)
矩阵 GA 中 1.0 的数量：184798


## 3.2.Home-brewing dataset

In [44]:
class myDataBase(Dataset):
    def __init__(self, hdf5_3D_Grids_file_path:str, hdf5_performance_file_path=1):
        '''
        name:  'len_{i}_sigma_{j}_temp_{k}'
        label: [len_i, sigma_i, temp_i], type: pytorch tensor
        structure: 3D grid, type: pytorch tensor
        '''
        # h5py file of 3D grids
        self.grids_hdf5     = h5py.File(hdf5_3D_Grids_file_path, 'r')
        # performance data file
        # TODO: h5py_performance_file_path

        # Sorted name list of all GA structures
        self.data_NameList  = self.sort_names(list(self.grids_hdf5.keys()))

    def __len__(self):
        return len(self.data_NameList)

    def __getitem__(self, index):
        # 解析: index -> label
        name_i      = self.data_NameList[index]
        label_i     = self.extract_label_from_key(name_i) # 【len_i, sigma_i, temp_i

        # 提取：hdf5 => np.ndarray
        grid_i      = self.grids_hdf5[name_i][:]

        # 转换: np.ndarray -> PyTorch tensor
        grid_i      = torch.from_numpy(grid_i)

        # 返回： 数据，标签
        return grid_i, label_i

    def close(self):
         # 关闭h5文件，防止损坏
        self.grids_hdf5.close()

    def extract_label_from_key(self, name:str):
        # 解析数据点名称为三维张量
        len_val   = int(name.split('_')[1])
        sigma_val = int(name.split('_')[3])
        temp_val  = int(name.split('_')[-1])
        key       = [len_val, sigma_val, temp_val]
        return torch.tensor(key)

    def sort_names(self, strings:str):
        # 数据点列表名称排序
        def key_func(s):
            match = re.match(r"len_(\d+)_sigma_(\d+)_(\d+)", s)  # 提取数字
            if match:
                len_val, sigma_val, temp_val = map(int, match.groups())
                return (len_val, sigma_val, -temp_val)  # 第三个数字取反，实现降序
            else:
                return (float('inf'), float('inf'), float('-inf'))  # 处理不匹配的情况
        return sorted(strings, key=key_func)  # 排序


# 使用方法
dataset         = myDataBase(conf.DB_3D_Grids_path)
data_i, label_i = dataset[0]
print(data_i.shape, label_i)
dataset.close() # 关闭读取

<class 'h5py._hl.base.KeysViewHDF5'>
torch.Size([400, 400, 400]) tensor([  2,   8, 373])


In [46]:
# 查看文件名列表
for i in dataset.data_NameList:
    print(i)
dataset.close() # 关闭读取

len_2_sigma_8_373
len_2_sigma_8_363
len_2_sigma_8_353
len_2_sigma_8_343
len_2_sigma_9_373
len_2_sigma_9_363
len_2_sigma_9_353
len_2_sigma_9_343
len_2_sigma_10_373
len_2_sigma_10_363
len_2_sigma_10_353
len_2_sigma_10_343
len_2_sigma_11_373
len_2_sigma_11_363
len_2_sigma_11_353
len_2_sigma_11_343
len_2_sigma_12_373
len_2_sigma_12_363
len_2_sigma_12_353
len_2_sigma_12_343
len_2_sigma_13_373
len_2_sigma_13_363
len_2_sigma_13_353
len_2_sigma_13_343
len_2_sigma_14_373
len_2_sigma_14_363
len_2_sigma_14_353
len_2_sigma_14_343
len_2_sigma_15_373
len_2_sigma_15_363
len_2_sigma_15_353
len_2_sigma_15_343
len_2_sigma_16_373
len_2_sigma_16_363
len_2_sigma_16_353
len_2_sigma_16_343
len_2_sigma_17_373
len_2_sigma_17_363
len_2_sigma_17_353
len_2_sigma_17_343
len_2_sigma_18_373
len_2_sigma_18_363
len_2_sigma_18_353
len_2_sigma_18_343
len_3_sigma_8_373
len_3_sigma_8_363
len_3_sigma_8_353
len_3_sigma_8_343
len_3_sigma_9_373
len_3_sigma_9_363
len_3_sigma_9_353
len_3_sigma_9_343
len_3_sigma_10_373
len_3_sig